In [6]:
import pandas as pd

In [7]:
df1 = pd.read_csv('rice_yield_22-23.csv')

In [8]:
df2 = pd.read_csv('bharatviz-district-template.csv')

In [9]:
df1

,State,District,Yield (Ton./Ha.)
0,Andaman and Nicobar Islands,Nicobars,1.26
1,Andaman and Nicobar Islands,North and middle andaman,2.11
2,Andaman and Nicobar Islands,South andamans,2.08
3,Andhra Pradesh,Alluri sitharama raju,2.99
4,Andhra Pradesh,Anakapalli,3.14
...,...,...,...
671,West Bengal,Murshidabad,3.13
672,West Bengal,Nadia,3.24
673,West Bengal,Paschim bardhaman,2.91
674,West Bengal,Purba bardhaman,3.15


In [10]:
df2

,state_name,district_name,some column name
0,A & N Islands,Nicobars,NaN
1,A & N Islands,North And Middle Andaman,NaN
2,A & N Islands,South Andamans,NaN
3,Andhra Pradesh,Alluri Sitharama Raju,NaN
4,Andhra Pradesh,Anakapalli,NaN
...,...,...,...
780,West Bengal,Murshidabad,NaN
781,West Bengal,Nadia,NaN
782,West Bengal,Paschim Bardhaman,NaN
783,West Bengal,Purba Bardhaman,NaN


In [17]:
import pandas as pd

# Load your data
# df1 = pd.read_csv('actual_data.csv')
# df2 = pd.read_csv('template.csv')

# ===============================
# 🔧 Step 1: Define manual mappings
# ===============================

# Manual state name corrections (lowercased)
state_name_mapping = {
    "andaman and nicobar islands": "a & n islands",
}

# Manual district name corrections (lowercased)
district_name_mapping = {
    # Example:
    # "north and middle andaman": "north and middle andaman",
}

# ===============================
# 🧼 Step 2: Normalization function
# ===============================

def normalize_name(name, mapping=None):
    if pd.isna(name):
        return ""
    name = str(name).lower().strip()
    name = name.replace("&", "and").replace(".", "").replace("-", " ")
    name = ' '.join(name.split())  # remove double spaces
    if mapping and name in mapping:
        return mapping[name]
    return name

# ===============================
# 🧹 Step 3: Apply normalization
# ===============================

# Normalize df1 (actual data)
df1['state_key'] = df1['State'].apply(lambda x: normalize_name(x, state_name_mapping))
df1['district_key'] = df1['District'].apply(lambda x: normalize_name(x, district_name_mapping))

# Normalize df2 (template)
df2['state_key'] = df2['state_name'].apply(lambda x: normalize_name(x, state_name_mapping))
df2['district_key'] = df2['district_name'].apply(lambda x: normalize_name(x, district_name_mapping))

# ===============================
# 🔗 Step 4: Merge
# ===============================

merged = df2.merge(
    df1[['state_key', 'district_key', 'Yield (Ton./Ha.)']],
    on=['state_key', 'district_key'],
    how='left'
)

# ===============================
# 📝 Step 5: Insert yield into template column
# ===============================

# If template has a column like 'some column name', replace it
merged['some column name'] = merged['Yield (Ton./Ha.)']
merged.drop(columns=['Yield (Ton./Ha.)', 'state_key', 'district_key'], inplace=True)

merged

,state_name,district_name,some column name
0,A & N Islands,Nicobars,NaN
1,A & N Islands,North And Middle Andaman,NaN
2,A & N Islands,South Andamans,NaN
3,Andhra Pradesh,Alluri Sitharama Raju,2.99
4,Andhra Pradesh,Anakapalli,3.14
...,...,...,...
780,West Bengal,Murshidabad,3.13
781,West Bengal,Nadia,3.24
782,West Bengal,Paschim Bardhaman,2.91
783,West Bengal,Purba Bardhaman,3.15


In [19]:
# Perform merge with indicator to track merge status
merged_debug = df2.merge(
    df1[['state_key', 'district_key', 'Yield (Ton./Ha.)']],
    on=['state_key', 'district_key'],
    how='left',
    indicator=True
)

# Filter rows that didn't find a match in df1
unmatched = merged_debug[merged_debug['_merge'] == 'left_only']

# Display unmatched rows (only those that failed to match)
print("Truly unmatched rows:")
print(unmatched[['state_name', 'district_name']].drop_duplicates())


Truly unmatched rows:
            state_name             district_name
0        A & N Islands                  Nicobars
1        A & N Islands  North And Middle Andaman
2        A & N Islands            South Andamans
28      Andhra Pradesh                    Y.s.r.
34   Arunachal Pradesh  Itanagar Capital Complex
..                 ...                       ...
665          Telangana       Yadadri Bhuvanagiri
680      Uttar Pradesh                   Ayodhya
690      Uttar Pradesh                   Bhadohi
734      Uttar Pradesh                 Prayagraj
776        West Bengal                   Kolkata

[157 rows x 2 columns]


In [25]:
from rapidfuzz import process, fuzz
import pandas as pd

# Step 0: (Assume you’ve already loaded and normalized df1 and df2 as before)

# Step 1: Merge with indicator to get unmatched rows
merged_debug = df2.merge(
    df1[['state_key', 'district_key', 'Yield (Ton./Ha.)']],
    on=['state_key', 'district_key'],
    how='left',
    indicator=True
)

unmatched = merged_debug[merged_debug['_merge'] == 'left_only']
unmatched = unmatched[['state_name', 'district_name', 'state_key', 'district_key']].drop_duplicates()

# Step 2: Create list of valid keys from df1 for comparison
valid_keys = df1[['state_key', 'district_key']].drop_duplicates()

# Step 3: Use fuzzy matching to suggest the best match for each unmatched row
suggested_mappings = []

for _, row in unmatched.iterrows():
    u_state = row['state_key']
    u_district = row['district_key']

    # Filter to only state_key matches to narrow down
    state_filtered = valid_keys[valid_keys['state_key'] == u_state]
    
    if not state_filtered.empty:
        # Match only within the same state
        match, score, _ = process.extractOne(
            u_district,
            state_filtered['district_key'],
            scorer=fuzz.token_sort_ratio
        )
        best_match_row = state_filtered[state_filtered['district_key'] == match].iloc[0]
        suggested_mappings.append({
            'original_state': row['state_name'],
            'original_district': row['district_name'],
            'suggested_state_key': best_match_row['state_key'],
            'suggested_district_key': best_match_row['district_key'],
            'similarity': score
        })
    else:
        suggested_mappings.append({
            'original_state': row['state_name'],
            'original_district': row['district_name'],
            'suggested_state_key': None,
            'suggested_district_key': None,
            'similarity': 0
        })

# Step 4: Create DataFrame from suggestions
suggestions_df = pd.DataFrame(suggested_mappings)
suggestions_df = suggestions_df.sort_values(by='similarity', ascending=False)

# Optional: Save to CSV to review manually
suggestions_df.to_csv('suggested_district_mappings.csv', index=False)

# Show top suggestions
print(suggestions_df.head(20))


    original_state         original_district suggested_state_key  \
64       Karnataka           Chamarajanagara           karnataka   
147      Telangana              Mahabubnagar           telangana   
149      Telangana                Narayanpet           telangana   
60       Karnataka                 Bagalkote           karnataka   
22         Gujarat                   Arvalli             gujarat   
139     Tamil Nadu                   Tenkasi          tamil nadu   
67       Karnataka          Dakshina Kannada           karnataka   
72       Karnataka            Uttara Kannada           karnataka   
146      Telangana    Kumuram Bheem Asifabad           telangana   
104         Punjab                 Ferozepur              punjab   
65       Karnataka           Chikkaballapura           karnataka   
66       Karnataka            Chikkamagaluru           karnataka   
63       Karnataka           Bengaluru Rural           karnataka   
151      Telangana               Ranga Reddy    

In [ ]:
merged.to_csv('basic_matching_output.csv', index=False)